# Anatomy of a Simulation

Welcome to the Smart Water Platform! This notebook is the first in a series designed to introduce you to the core concepts of building and running simulations. Here, we will dissect the fundamental components of the simulation engine: the `SimulationHarness` and the `MessageBus`.

## The SimulationHarness: The Conductor of the Orchestra

The `SimulationHarness` is the central coordinator for any simulation. Its primary job is to manage all the different components of a water system, understand how they are connected, and execute the simulation step-by-step in the correct order.

Setting up a simulation involves a few key steps with the harness:

### 1. Initialization
First, we create an instance of the harness, giving it a basic configuration dictionary that specifies the simulation's `duration` and the time step `dt` (both in seconds).

In [ ]:
from swp.core_engine.testing.simulation_harness import SimulationHarness

simulation_config = {'duration': 600, 'dt': 1.0}
harness = SimulationHarness(config=simulation_config)

### 2. Adding Components
Next, we create instances of our physical components (like reservoirs, pipes, and gates) and add them to the harness using the `add_component` method. Each component must have a unique name.

In [ ]:
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.simulation_identification.physical_objects.pipe import Pipe

# Define the components
reservoir = Reservoir(name="res1", initial_state={'volume': 25e6, 'water_level': 15.0}, parameters={'surface_area': 1.5e6})
pipe = Pipe(name="pipe1", initial_state={'outflow': 0}, parameters={'length': 1000, 'diameter': 1.5, 'friction_factor': 0.02})
gate = Gate(name="g1", initial_state={'opening': 0.3}, parameters={'width': 10, 'max_opening': 1.0, 'discharge_coefficient': 0.6})

# Add them to the harness
harness.add_component(reservoir)
harness.add_component(pipe)
harness.add_component(gate)

### 3. Defining the Topology
Once the components are added, we tell the harness how they are connected using `add_connection`. This defines the network's **topology** (the directed graph of how water flows). The harness stores this topology to determine the simulation order.

In [ ]:
harness.add_connection("res1", "pipe1")
harness.add_connection("pipe1", "g1")

### 4. Visualizing the Topology
The harness stores the connections in its `topology` attribute. We can use a library like `NetworkX` to quickly visualize the system we've just defined.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# The harness topology is a dictionary suitable for creating a directed graph
G = nx.DiGraph(harness.topology)

plt.figure(figsize=(8, 4))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_size=3000, node_color='lightblue', font_size=12, font_weight='bold', arrows=True, arrowsize=20)
plt.title('Simulation Network Topology')
plt.show()

### 5. Building and Running
- **`build()`**: This crucial method prepares the harness for execution. Its main job is to perform a **topological sort** on the component graph. This ensures that when the simulation runs, upstream components are always calculated before their downstream counterparts.
- **`run_simulation()`**: This method executes the main simulation loop. At each time step, it iterates through the sorted components, calling their `step` methods with the appropriate inputs. 

After running, the results are stored in the `harness.history` attribute, which we can use for analysis and plotting.

In [ ]:
# Build and run the simulation
# We redirect the verbose simulation log to a file to keep the notebook output clean
import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    harness.run_simulation()
sys.stdout = original_stdout # Restore stdout

print("Simulation complete. Results are stored in harness.history.")

## The MessageBus: The Nervous System

You may have noticed that the `SimulationHarness` also creates a `message_bus`. While not used in this simple, centralized simulation, the message bus is the backbone of all agent-based control. It allows components and agents to communicate in a decoupled way by publishing and subscribing to messages on named 'topics'.

This powerful concept will be the focus of our next notebook, **"Guide to Feedback Control"**.

## Analyzing the Results

Finally, we can inspect the `harness.history` to see what happened during the simulation. Let's plot the water level in the reservoir.

In [ ]:
import pandas as pd

time = [h['time'] for h in harness.history]
res_level = [h['res1']['water_level'] for h in harness.history]

df = pd.DataFrame({'Time (s)': time, 'Reservoir Level (m)': res_level})

df.plot(x='Time (s)', y='Reservoir Level (m)', figsize=(12, 6), grid=True, title='Reservoir Water Level During Simulation')